In [1]:
import pandas as pd

In [2]:
#reddit_file = 'running.csv'
reddit_file = 'AskCulinary.csv'


In [4]:
reddit_posts = pd.read_csv(f'./data/{reddit_file}')

In [5]:
reddit_posts.tail()

,subreddit,selftext,title,author,created_utc,created_date
1095,AskCulinary,I'm really having a hard time being creative a...,What Would You Make For Christmas Eve Dinner F...,mattyjayy,1608153106,2020-12-16 21:11:46
1096,AskCulinary,"Ho ho ho, fellow redditers,\n\nMy girlfriend a...",Need help with preparing a Christmas dinner th...,AceOblivion,1608153000,2020-12-16 21:10:00
1097,AskCulinary,I'm usually not too picky and I'm happy with m...,When should I pull my steaks off for rare/med ...,blakjack9,1608152777,2020-12-16 21:06:17
1098,AskCulinary,I was considering roasting a chicken for Chris...,Chicken roasting temp versus turkey roasting temp,I_Like_Knitting_TBH,1608151957,2020-12-16 20:52:37
1099,AskCulinary,"It seems like traditionally, cream isn’t used....",Why don’t traditional pastas use cream?,ScruffyMayfield,1608149939,2020-12-16 20:18:59


In [6]:
reddit_posts.isnull().sum()

subreddit        0
selftext        10
title            0
author           0
created_utc      0
created_date     0
dtype: int64

In [7]:
reddit_posts.dropna(inplace=True)
reddit_posts.reset_index(inplace=True)

In [8]:
reddit_posts.isnull().sum()

index           0
subreddit       0
selftext        0
title           0
author          0
created_utc     0
created_date    0
dtype: int64